In [ ]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# STEP 2: Set the dataset directory from Google Drive
data_dir = '/content/drive/MyDrive/Data'  # <-- Update with your path

In [ ]:
# STEP 3: Import Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model

In [ ]:
# STEP 4: Image Preprocessing and Loading
img_size = 224
batch_size = 32

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

Found 13926 images belonging to 16 classes.


In [ ]:
# STEP 5: Feature Extraction using MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3), pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Generate features
features = model.predict(generator, verbose=1)
labels = generator.classes
class_names = list(generator.class_indices.keys())

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


436/436 ━━━━━━━━━━━━━━━━━━━━ 5635s 13s/step


In [7]:
# STEP 6: Save Features to Drive (Optional, Speeds up Re-runs)
np.save('/content/drive/MyDrive/Models/features.npy', features)
np.save('/content/drive/MyDrive/Models/labels.npy', labels)

In [8]:
# STEP 7: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [9]:
# STEP 8: Train Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
# STEP 9: Evaluate
y_pred = rf.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=class_names))

✅ Accuracy: 0.9339554917444365
                       precision    recall  f1-score   support

     Bacterial_blight       0.99      0.92      0.95       107
    Bacterial_pustule       0.86      0.94      0.90        97
          Caterpillar       0.82      0.63      0.71       106
  Diabrotica_speciosa       0.92      0.63      0.75        93
         Downy_mildew       0.96      0.88      0.92       105
             Ferrugen       1.00      0.98      0.99        92
    Frogeye_Leaf_Spot       0.96      0.78      0.86       101
              Healthy       0.93      0.99      0.96      1299
        Mossaic_Virus       0.99      1.00      0.99        96
       Powdery_mildew       0.98      0.95      0.96        93
                 Rust       0.91      0.90      0.90       115
             Septoria       1.00      0.90      0.95        94
      Southern_blight       0.88      1.00      0.94       100
Sudden_Death_Syndrome       0.97      1.00      0.98        92
     Target_Leaf_Spot  

In [16]:
import json

class_names = list(generator.class_indices.keys())  # After training
with open('/content/drive/MyDrive/Models/class_names.json', 'w') as f:
    json.dump(class_names, f)

In [11]:
import joblib
joblib.dump(rf, '/content/drive/MyDrive/Models/rf_model.pkl')

['/content/drive/MyDrive/Models/rf_model.pkl']

In [12]:
rf = joblib.load('/content/drive/MyDrive/Models/rf_model.pkl')

In [13]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

img_path = '/content/drive/MyDrive/Full_Data/Ferrugen/image_41.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

In [14]:
feature = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [15]:
predicted_class_index = rf.predict(feature)[0]
predicted_class_name = class_names[predicted_class_index]
print("🔍 Predicted Class:", predicted_class_name)

🔍 Predicted Class: Ferrugen


In [ ]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# STEP 2: Set the dataset directory from Google Drive
data_dir = '/content/drive/MyDrive/Data'  # <-- Update with your path
# STEP 3: Import Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
# STEP 4: Image Preprocessing and Loading
img_size = 224
batch_size = 32

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)
# STEP 5: Feature Extraction using MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3), pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)
# Generate features
features = model.predict(generator, verbose=1)
labels = generator.classes
class_names = list(generator.class_indices.keys())
# STEP 6: Save Features to Drive (Optional, Speeds up Re-runs)
np.save('/content/drive/MyDrive/Models/features.npy', features)
np.save('/content/drive/MyDrive/Models/labels.npy', labels)
# STEP 7: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
# STEP 8: Train Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
# STEP 9: Evaluate
y_pred = rf.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
# from IPython import get_ipython
# from IPython.display import display
# # %%
# # STEP 1: Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# # %%
# # STEP 2: Set the dataset directory from Google Drive
# data_dir = '/content/drive/MyDrive/Data'  # <-- Update with your path
# # %%
# # STEP 3: Import Libraries
# import numpy as np
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.models import Model
# # %%
# # STEP 4: Image Preprocessing and Loading
# img_size = 224
# batch_size = 32

# # Create ImageDataGenerator with augmentation
# datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2  # Split data into train and validation sets
# )

# # Create data generators for train and validation sets
# train_generator = datagen.flow_from_directory(
#     data_dir,
#     target_size=(img_size, img_size),
#     batch_size=batch_size,
#     class_mode='sparse',
#     subset='training',  # Use training subset
#     shuffle=True
# )

# validation_generator = datagen.flow_from_directory(
#     data_dir,
#     target_size=(img_size, img_size),
#     batch_size=batch_size,
#     class_mode='sparse',
#     subset='validation',  # Use validation subset
#     shuffle=False
# )
# # %%
# # STEP 5: Feature Extraction using MobileNetV2
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3), pooling='avg')
# model = Model(inputs=base_model.input, outputs=base_model.output)
# # %%
# # Generate features for training set
# train_features = model.predict(train_generator, verbose=1)
# train_labels = train_generator.classes

# # Generate features for validation set
# validation_features = model.predict(validation_generator, verbose=1)
# validation_labels = validation_generator.classes
# # %%
# # STEP 6: Save Features to Drive (Optional, Speeds up Re-runs)
# np.save('/content/drive/MyDrive/Models/train_features.npy', train_features)
# np.save('/content/drive/MyDrive/Models/train_labels.npy', train_labels)
# np.save('/content/drive/MyDrive/Models/validation_features.npy', validation_features)
# np.save('/content/drive/MyDrive/Models/validation_labels.npy', validation_labels)
# # %%
# # STEP 7: Train Random Forest Classifier
# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf.fit(train_features, train_labels)
# # %%
# # STEP 8: Evaluate
# y_pred = rf.predict(validation_features)
# class_names = list(train_generator.class_indices.keys())
# print("✅ Accuracy:", accuracy_score(validation_labels, y_pred))
# print(classification_report(validation_labels, y_pred, target_names=class_names))
# # %%